## == Point to Polygoan == 

The script requires the following libraries: pandas and geopandas.
You can install them using pip with the following commands:

pip install pandas
pip install geopandas 

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def create_dissolved_polygons_from_points(input_csv_path, output_shp_path, class_id_column, buffer_distance, x_col, y_col):
    
    """Creates polygons around points from a CSV, dissolving them based on a class ID.

    Args:
        input_csv_path (str): "/home/dibakar88/N24E90_Training_poly_Brick/N24E90.csv"
        output_shp_path (str): "/home/dibakar88/N24E90_Training_poly_Brick/output"
        class_id_column (str): "class_ID"
        buffer_distance (float): 10.
        x_col (str): "Lon"
        y_col (str): "Lat"    
    """
          
    try:
        # 1. Load the input CSV file into a pandas DataFrame
        print(f"Loading input CSV file: {input_csv_path}")
        df = pd.read_csv(input_csv_path)
        print("CSV loaded successfully.")

        # Check if the class ID and coordinate columns exist
        if class_id_column not in df.columns or x_col not in df.columns or y_col not in df.columns:
            raise ValueError("Required columns are missing from the CSV file. "
                             f"Please check for '{class_id_column}', '{x_col}', and '{y_col}'.")

        # 2. Convert the DataFrame to a GeoDataFrame
        print("Converting DataFrame to GeoDataFrame...")
        # Create a list of shapely Point objects from the x and y columns
        geometry = [Point(xy) for xy in zip(df[x_col], df[y_col])]
        # Create the GeoDataFrame, specifying the geometry and original CRS
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # CRS is set to WGS84 for lat/lon

        # 3. Create a buffer around each point
        print(f"Creating buffers of {buffer_distance} units around each point...")
        # A new GeoDataFrame is created with the buffered geometries
        gdf['geometry'] = gdf.geometry.buffer(buffer_distance)
        
        # 4. Dissolve the buffered polygons based on the class ID column
        print(f"Dissolving buffers based on the '{class_id_column}' column...")
        # The dissolve operation merges all geometries with the same value in the specified column.
        dissolved_gdf = gdf.dissolve(by=class_id_column)
        print("Dissolve complete.")

        # 5. Save the resulting GeoDataFrame to a new shapefile
        print(f"Saving dissolved polygons to: {output_shp_path}")
        dissolved_gdf.to_file(output_shp_path)
        print("Process completed successfully!")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage:
if __name__ == "__main__":
    # Define your input and output paths, class ID column, and buffer distance
    # NOTE: Replace these with your actual file paths and parameters.
    INPUT_FILE = "/home/dibakar88/N24E90_Training_poly_Brick/N24E90.csv"
    OUTPUT_FILE = "/home/dibakar88/N24E90_Training_poly_Brick/output.shp"
    CLASS_COLUMN = "class_ID"
    BUFFER_SIZE = 0.0005
    X_COLUMN = "Lon"
    Y_COLUMN = "Lat"  
    
    
    # Call the main function to run the process
    create_dissolved_polygons_from_points(INPUT_FILE, OUTPUT_FILE, CLASS_COLUMN, BUFFER_SIZE, X_COLUMN, Y_COLUMN)


Loading input CSV file: /home/dibakar88/N24E90_Training_poly_Brick/N24E90.csv
CSV loaded successfully.
Converting DataFrame to GeoDataFrame...
Creating buffers of 0.0005 units around each point...


/tmp/ipykernel_3880501/956281545.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf.geometry.buffer(buffer_distance)


Dissolving buffers based on the 'class_ID' column...
Dissolve complete.
Saving dissolved polygons to: /home/dibakar88/N24E90_Training_poly_Brick/output.shp
Process completed successfully!


/home/dibakar88/anaconda3/envs/myenv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Unnamed: 0' to 'Unnamed_ 0'
  ogr_write(
